---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

Python Pickle is used to serialize and deserialize a python object structure. Any object on python can be pickled so that it can be saved on disk.
The primary argument of pickle load function is the file object that you get by opening the file in read-binary (rb) mode.

In [2]:
P1_Graphs = pickle.load(open('A4_graphs', 'rb'))
P1_Graphs

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [3]:
def graph_identification():
    
    # PA: small shortest path, very small clustering coeff
    # Small world model: small shortest path, high clustering coeff
    # SW_L : low p has small shortest path and high clustering coeff
    # SW_H : high p has small shortest path decrease rapily and clustering coeff decrease slowly
    
    parameters =[(nx.average_shortest_path_length(G), nx.average_clustering(G)) for G in P1_Graphs]
    
    return ['PA', 'SW_L', 'SW_L', 'PA', 'SW_H'] # parameters 
graph_identification() 

['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [4]:
G = nx.read_gpickle('email_prediction.txt')

print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [5]:
def salary_predictions():
    
    df= pd.DataFrame(index= G.nodes())
    df['Department']= pd.Series(nx.get_node_attributes(G, 'Department'))
    df['ManagementSalary']= pd.Series(nx.get_node_attributes(G, 'ManagementSalary'))
    df['clustering'] = pd.Series(nx.clustering(G))
    df['degree centrality']= pd.Series(nx.degree_centrality(G)) #many connections
    df['closeness']= pd.Series(nx.closeness_centrality(G)) #close to other nodes
    df['betweenness']= pd.Series(nx.betweenness_centrality(G, normalized= True, endpoints=False))

    X_data = df.dropna().drop('ManagementSalary', axis=1)
    y_data= df.dropna()['ManagementSalary']

    X_predict= df[df['ManagementSalary'].isnull()].drop('ManagementSalary', axis=1)

    from sklearn.model_selection import train_test_split
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.metrics import roc_auc_score
    X_train, X_test, y_train, y_test= train_test_split(X_data, y_data, random_state=0)
    
    clf= RandomForestClassifier(n_estimators=7, random_state=0).fit(X_train, y_train)
    y_predict= clf.predict(X_test)
#     print('AUC: ', roc_auc_score(y_test, y_predict))
#     print('Train score ', clf.score(X_train, y_train))
#     print('Test score ', clf.score(X_test,y_test))

    salary_clf= RandomForestClassifier(n_estimators=7, random_state=0).fit(X_data, y_data)
    salary_predict= salary_clf.predict_proba(X_predict)[:,1]
  
    return pd.Series(salary_predict, index= X_predict.index)
salary_predictions()

1       0.000000
2       1.000000
5       1.000000
8       0.142857
14      0.142857
          ...   
992     0.000000
994     0.000000
996     0.000000
1000    0.000000
1001    0.142857
Length: 252, dtype: float64

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [6]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [8]:
def new_connections_predictions():
    
    future_connections['jaccard coef']= [i[2] for i in nx.jaccard_coefficient(G, future_connections.index)]
    future_connections['preferential attachment']= [i[2] for i in nx.preferential_attachment(G, future_connections.index)]    
    future_connections['community common neighbors']= [i[2] for i in nx.cn_soundarajan_hopcroft(G, future_connections.index, community='Department')]
    future_connections['community resource allocation']= [i[2] for i in nx.ra_index_soundarajan_hopcroft(G, future_connections.index, community='Department')]

    X_data= future_connections.dropna().drop('Future Connection', axis=1)
    y_data= future_connections.dropna()['Future Connection']
    X_predict= future_connections[future_connections['Future Connection'].isnull()].drop('Future Connection', axis=1)

    from sklearn.model_selection import train_test_split
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.metrics import roc_auc_score
    
    X_train, X_test, y_train, y_test= train_test_split(X_data,y_data, random_state=0)
    clf= DecisionTreeClassifier(max_depth=5).fit(X_train,y_train)
    y_predict= clf.predict(X_test)
#     print('AUC: ', roc_auc_score(y_test, y_predict))
#     print('Train score ', clf.score(X_train, y_train))
#     print('Test score ', clf.score(X_test,y_test))

    connections_clf= DecisionTreeClassifier(max_depth=5).fit(X_data, y_data)
    connections_pred= connections_clf.predict_proba(X_predict)[:,1]
    return pd.Series(connections_pred, index= X_predict.index)

new_connections_predictions()

(107, 348)    0.037441
(542, 751)    0.012713
(20, 426)     0.518100
(50, 989)     0.012713
(942, 986)    0.012713
                ...   
(165, 923)    0.012713
(673, 755)    0.012713
(939, 940)    0.012713
(555, 905)    0.012713
(75, 101)     0.016470
Length: 122112, dtype: float64

<img src= 'assignment 4 grade.jpg' />